In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [19]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd, pickup_location_hex_8,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230402'
and yyyymmdd <= '20230409'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [20]:
#Create a copy of the df retrived from presto
df = df_copy.copy()

In [21]:
#View the dataset
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,pickup_location_hex_8,order_status,cancel_reason
0,643170d8c0b4c4657b763651,NaN,8861892e9bfffff,12.981025,77.610001,Bangalore,8861892e9bfffff,12.981257,77.610176,191912,19,1915,Auto,19:00,6,20230408,8861892e9bfffff,customerCancelled,order cancelled before rider accepted
1,643170dc51c2a20449402e01,NaN,8860145841fffff,12.976253,77.529760,Bangalore,8860145841fffff,12.976315,77.529759,191916,19,1915,Auto,19:00,6,20230408,8860145841fffff,customerCancelled,order cancelled before rider accepted
2,643170f7260d997daf4f0bde,NaN,88618920c1fffff,12.923882,77.736627,Bangalore,88618920c7fffff,12.933841,77.736847,191943,19,1915,Auto,19:00,6,20230408,88618920c7fffff,expired,None
3,643171326b042d72ccd0bea3,0.922,8861892dddfffff,13.071511,77.637932,Bangalore,8861892ce9fffff,13.045229,77.626916,192042,19,1915,Auto,19:00,6,20230408,8861892ce9fffff,dropped,None
4,6431713c233d313201965842,NaN,88618925d7fffff,12.924501,77.597058,Bangalore,88618925d1fffff,12.917089,77.596870,192052,19,1915,Auto,19:00,6,20230408,88618925d1fffff,expired,None


In [22]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,pickup_location_hex_8,order_status,cancel_reason,second,minute,date
0,643170d8c0b4c4657b763651,NaN,8861892e9bfffff,12.981025,77.610001,Bangalore,8861892e9bfffff,12.981257,77.610176,191912,19,1915,Auto,19:00,6,20230408,8861892e9bfffff,customerCancelled,order cancelled before rider accepted,12,19,08
1,643170dc51c2a20449402e01,NaN,8860145841fffff,12.976253,77.529760,Bangalore,8860145841fffff,12.976315,77.529759,191916,19,1915,Auto,19:00,6,20230408,8860145841fffff,customerCancelled,order cancelled before rider accepted,16,19,08
2,643170f7260d997daf4f0bde,NaN,88618920c1fffff,12.923882,77.736627,Bangalore,88618920c7fffff,12.933841,77.736847,191943,19,1915,Auto,19:00,6,20230408,88618920c7fffff,expired,None,43,19,08
3,643171326b042d72ccd0bea3,0.922,8861892dddfffff,13.071511,77.637932,Bangalore,8861892ce9fffff,13.045229,77.626916,192042,19,1915,Auto,19:00,6,20230408,8861892ce9fffff,dropped,None,42,20,08
4,6431713c233d313201965842,NaN,88618925d7fffff,12.924501,77.597058,Bangalore,88618925d1fffff,12.917089,77.596870,192052,19,1915,Auto,19:00,6,20230408,88618925d1fffff,expired,None,52,20,08


In [23]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,pickup_location_hex_8,order_status,cancel_reason,second,minute,date,20_sec_batch,count
0,643170d8c0b4c4657b763651,NaN,8861892e9bfffff,12.981025,77.610001,Bangalore,8861892e9bfffff,12.981257,77.610176,191912,19,1915,Auto,19:00,6,20230408,8861892e9bfffff,customerCancelled,order cancelled before rider accepted,12.0,19.0,08,0-20,1
1,643170dc51c2a20449402e01,NaN,8860145841fffff,12.976253,77.529760,Bangalore,8860145841fffff,12.976315,77.529759,191916,19,1915,Auto,19:00,6,20230408,8860145841fffff,customerCancelled,order cancelled before rider accepted,16.0,19.0,08,0-20,1
2,643170f7260d997daf4f0bde,NaN,88618920c1fffff,12.923882,77.736627,Bangalore,88618920c7fffff,12.933841,77.736847,191943,19,1915,Auto,19:00,6,20230408,88618920c7fffff,expired,None,43.0,19.0,08,40-60,1
3,643171326b042d72ccd0bea3,0.922,8861892dddfffff,13.071511,77.637932,Bangalore,8861892ce9fffff,13.045229,77.626916,192042,19,1915,Auto,19:00,6,20230408,8861892ce9fffff,dropped,None,42.0,20.0,08,40-60,1
4,6431713c233d313201965842,NaN,88618925d7fffff,12.924501,77.597058,Bangalore,88618925d1fffff,12.917089,77.596870,192052,19,1915,Auto,19:00,6,20230408,88618925d1fffff,expired,None,52.0,20.0,08,40-60,1


In [24]:
#Check the number of rides coming from each hex
df_grouped = df.groupby(['pickup_location_hex_8'])['count'].sum().reset_index()

#Filter out those hexes where atleast 1000 rides have come in the last two weeks
df_grouped = df_grouped[df_grouped['count'] >= 100].reset_index(drop = True)

#Add a column to keep for analysis
df_grouped['keep_for_analysis'] = 'yes'
df_grouped.head()

,pickup_location_hex_8,count,keep_for_analysis
0,8860144b4dfffff,733,yes
1,8860145109fffff,122,yes
2,8860145121fffff,670,yes
3,8860145123fffff,604,yes
4,8860145125fffff,841,yes


In [25]:
#Merge the number of rides to the original df
df = df.merge(df_grouped[['pickup_location_hex_8', 'keep_for_analysis']], on = 'pickup_location_hex_8', how = 'left')
df['keep_for_analysis'] = df['keep_for_analysis'].fillna('no')
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,pickup_location_hex_8,order_status,cancel_reason,second,minute,date,20_sec_batch,count,keep_for_analysis
0,643170d8c0b4c4657b763651,NaN,8861892e9bfffff,12.981025,77.610001,Bangalore,8861892e9bfffff,12.981257,77.610176,191912,19,1915,Auto,19:00,6,20230408,8861892e9bfffff,customerCancelled,order cancelled before rider accepted,12.0,19.0,08,0-20,1,yes
1,643170dc51c2a20449402e01,NaN,8860145841fffff,12.976253,77.529760,Bangalore,8860145841fffff,12.976315,77.529759,191916,19,1915,Auto,19:00,6,20230408,8860145841fffff,customerCancelled,order cancelled before rider accepted,16.0,19.0,08,0-20,1,yes
2,643170f7260d997daf4f0bde,NaN,88618920c1fffff,12.923882,77.736627,Bangalore,88618920c7fffff,12.933841,77.736847,191943,19,1915,Auto,19:00,6,20230408,88618920c7fffff,expired,None,43.0,19.0,08,40-60,1,yes
3,643171326b042d72ccd0bea3,0.922,8861892dddfffff,13.071511,77.637932,Bangalore,8861892ce9fffff,13.045229,77.626916,192042,19,1915,Auto,19:00,6,20230408,8861892ce9fffff,dropped,None,42.0,20.0,08,40-60,1,yes
4,6431713c233d313201965842,NaN,88618925d7fffff,12.924501,77.597058,Bangalore,88618925d1fffff,12.917089,77.596870,192052,19,1915,Auto,19:00,6,20230408,88618925d1fffff,expired,None,52.0,20.0,08,40-60,1,yes


In [26]:
#Filter the df only on hexes to keep
df_filter = df[df['keep_for_analysis'] == 'yes']

In [27]:
#Groupby on hex to see number of orders coming in 15 and 20 second batches
df_hex_grouped = df_filter.groupby(['pickup_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])['count'].sum().reset_index()
df_hex_grouped = df_hex_grouped.sort_values(by = ['pickup_location_hex_8', 'date', 'hour', 'minute', '20_sec_batch'])
df_hex_grouped.head()

,pickup_location_hex_8,date,hour,minute,20_sec_batch,count
0,8860144b4dfffff,02,00,0.0,0-20,0
1,8860144b4dfffff,02,00,0.0,20-40,0
2,8860144b4dfffff,02,00,0.0,40-60,0
3,8860144b4dfffff,02,00,1.0,0-20,0
4,8860144b4dfffff,02,00,1.0,20-40,0


In [28]:
#Identify high demand hexes
df_hex_grouped_filter2 = df_hex_grouped[df_hex_grouped['count'] >= 1]
df_hex_grouped_filter2['time_period'] = np.where(df_hex_grouped_filter2['hour'].isin(['17', '18', '19', '20', '21']), 'evening_peak',
                                                  np.where(df_hex_grouped_filter2['hour'].isin(['08', '09', '10', '11']), 'morning_peak', 'rest'))
count_df_evening = pd.DataFrame(df_hex_grouped_filter2[df_hex_grouped_filter2['time_period'] == 'evening_peak']['pickup_location_hex_8'].value_counts().reset_index())
count_df_evening.columns = ['pickup_location_hex_8', 'count']
count_df_evening.head(20)

,pickup_location_hex_8,count
0,88618925c7fffff,6150
1,8861892589fffff,5496
2,88618925c3fffff,5423
3,8861892421fffff,5396
4,8861892e9bfffff,5162
5,88618925e9fffff,4868
6,88618925cbfffff,4643
7,88618925d9fffff,4576
8,8861892553fffff,4561
9,88618925a5fffff,4559


In [29]:
count_df_morning = pd.DataFrame(df_hex_grouped_filter2[df_hex_grouped_filter2['time_period'] == 'morning_peak']['pickup_location_hex_8'].value_counts().reset_index())
count_df_morning.columns = ['pickup_location_hex_8', 'count']
count_df_morning.head(20)

,pickup_location_hex_8,count
0,88618925c3fffff,3778
1,88618925cbfffff,3520
2,88618925e9fffff,3458
3,8861892437fffff,3171
4,8861892589fffff,2998
5,88618925c1fffff,2950
6,88618920bdfffff,2501
7,8861892513fffff,2456
8,8861892433fffff,2453
9,88618925e1fffff,2450


In [85]:
new_df = count_df.head(20)
hexes = new_df['customer_location_hex_8'].tolist()

In [86]:
print(hexes)

['883da218c7fffff', '883da21889fffff', '883da21991fffff', '883da218bdfffff', '883da219b9fffff', '883da2198bfffff', '883da21999fffff', '883da219b3fffff', '883da219d7fffff', '883da218a5fffff', '883da20b59fffff', '883da21821fffff', '883da219a1fffff', '883da219d5fffff', '883da219ddfffff', '883da20a6dfffff', '883da218adfffff', '883da21995fffff', '883da219d3fffff', '883da2199dfffff']


In [33]:
df[['customer_location_hex_8', 'count']].to_csv('jaipur_hexes.csv', index = False)

In [56]:
hexes_kolkata = pd.read_csv('kolkata_hexes.csv')
hexes_kolkata_list = hexes_kolkata['pickup_location_hex_8'].unique().tolist()
hexes_kolkata_list

['883cf2c74bfffff',
 '883cf2c753fffff',
 '883cf2c629fffff',
 '883cf2c625fffff',
 '883cf2c751fffff',
 '883cf2c66bfffff',
 '883cf2c661fffff',
 '883cf2c667fffff',
 '883cf2c62dfffff',
 '883cf2c757fffff',
 '883cf2c645fffff',
 '883cf2c621fffff',
 '883cf2c647fffff',
 '883cf2c64dfffff',
 '883cf2c641fffff',
 '883cf2c601fffff',
 '883cf2c75bfffff',
 '883cf2c605fffff',
 '883cf2c627fffff',
 '883cf2c75dfffff']

In [58]:
#Look at the average demand for each minute
df_hex_grouped['time'] = (pd.to_datetime(df_hex_grouped['hour'].astype(str) + ':' + df_hex_grouped['minute'].astype(int).astype(str), format='%H:%M').dt.time)
df_hex_grouped.head()

,customer_location_hex_8,date,hour,minute,20_sec_batch,count,time
0,883cf21303fffff,06,00,0.0,0-20,0,00:00:00
1,883cf21303fffff,06,00,0.0,20-40,0,00:00:00
2,883cf21303fffff,06,00,0.0,40-60,0,00:00:00
3,883cf21303fffff,06,00,1.0,0-20,0,00:01:00
4,883cf21303fffff,06,00,1.0,20-40,0,00:01:00


In [57]:
#Filter on the hexes identified as high demand
high_demand_hexes = ['883da219ddfffff', '883da219d1fffff', '883da219d3fffff', '883da218a5fffff', '883da219d5fffff', '883da219d7fffff', '883da21999fffff',
 '883da2198bfffff', '883da2199dfffff', '883da219b9fffff', '883da21995fffff', '883da219b3fffff', '883da20a6dfffff', '883da219c7fffff']
hexes = []
for x in hexes_kolkata_list:
    hex_ids = h3.k_ring(x, 1)
    hexes.append(hex_ids)

hexes = [item for sublist in hexes for item in sublist]

In [64]:
len(set(hexes))

50

In [69]:
kolkata_hexes = pd.DataFrame(hexes)
kolkata_hexes.to_csv('kolkata_TAM.csv', index = False)

In [63]:
print(list(set(hexes)))

['883cf2c665fffff', '883cf2c641fffff', '883cf2c755fffff', '883cf2d5b5fffff', '883cf2c753fffff', '883cf2c757fffff', '883cf2c44dfffff', '883cf2c60dfffff', '883cf2c743fffff', '883cf2c71bfffff', '883cf2c66dfffff', '883cf2c66bfffff', '883cf2d59bfffff', '883cf2c64dfffff', '883cf2c75bfffff', '883cf2c629fffff', '883cf2c71dfffff', '883cf2c667fffff', '883cf2c663fffff', '883cf2d5b7fffff', '883cf2c64bfffff', '883cf2c669fffff', '883cf2c623fffff', '883cf2c609fffff', '883cf2c661fffff', '883cf2c601fffff', '883cf2c751fffff', '883cf2d4a7fffff', '883cf2c749fffff', '883cf2c74bfffff', '883cf2c719fffff', '883cf2c62dfffff', '883cf2c607fffff', '883cf2c647fffff', '883cf2c627fffff', '883cf2c603fffff', '883cf2c621fffff', '883cf2c70bfffff', '883cf2c605fffff', '883cf2c741fffff', '883cf2c649fffff', '883cf2c62bfffff', '883cf2c60bfffff', '883cf2c75dfffff', '883cf2c449fffff', '883cf2c759fffff', '883cf2c625fffff', '883cf2d593fffff', '883cf2c643fffff', '883cf2c645fffff']


In [59]:
#Get the total number of rides per hour
df_hex_high_demand = df_hex_grouped[df_hex_grouped['customer_location_hex_8'].isin(hexes)]
df_level_1_grouped = round(df_hex_high_demand.groupby(['date', 'hour'])['count'].sum().reset_index(), 0)
df_level_1_grouped = df_level_1_grouped.rename(columns = {'count':'rides_per_hour'})
df_level_1_grouped['rides_per_20_sec'] = df_level_1_grouped['rides_per_hour'].apply(lambda x: round(x/180, 2))
df_level_1_grouped

,date,hour,rides_per_hour,rides_per_20_sec
0,06,00,148,0.82
1,06,01,84,0.47
2,06,02,72,0.40
3,06,03,80,0.44
4,06,04,60,0.33
...,...,...,...,...
115,10,19,1164,6.47
116,10,20,988,5.49
117,10,21,729,4.05
118,10,22,449,2.49


In [37]:
df_hex_high_demand

,customer_location_hex_8,date,hour,minute,20_sec_batch,count,time
864000,883da20a61fffff,06,00,0.0,0-20,0,00:00:00
864001,883da20a61fffff,06,00,0.0,20-40,0,00:00:00
864002,883da20a61fffff,06,00,0.0,40-60,0,00:00:00
864003,883da20a61fffff,06,00,1.0,0-20,0,00:01:00
864004,883da20a61fffff,06,00,1.0,20-40,0,00:01:00
...,...,...,...,...,...,...,...
5075995,883da219ebfffff,10,23,58.0,20-40,0,23:58:00
5075996,883da219ebfffff,10,23,58.0,40-60,0,23:58:00
5075997,883da219ebfffff,10,23,59.0,0-20,0,23:59:00
5075998,883da219ebfffff,10,23,59.0,20-40,0,23:59:00


In [65]:
df[(df['hour'].isin(['17', '18', '19', '20', '21'])) & (df['pickup_location_hex_8'].isin(hexes))].shape[0]/df[df['hour'].isin(['17', '18', '19', '20', '21'])].shape[0]

0.30295429552840003

In [66]:
df[(df['hour'].isin(['08', '09', '10', '11'])) & (df['pickup_location_hex_8'].isin(hexes))].shape[0]/df[df['hour'].isin(['08', '09', '10', '11'])].shape[0]


0.21809916134644106

In [67]:
df[(df['hour'].isin(['08', '09', '10', '11'])) & (df['pickup_location_hex_8'].isin(hexes))].shape[0] + df[(df['hour'].isin(['17', '18', '19', '20', '21'])) & (df['pickup_location_hex_8'].isin(hexes))].shape[0]

37808

In [68]:
37808/df.shape[0]

0.15256911573026

In [70]:
jaipur_hexes = df_hex_high_demand['customer_location_hex_8'].unique().tolist()
print(jaipur_hexes)

['883cf2c449fffff', '883cf2c44dfffff', '883cf2c601fffff', '883cf2c603fffff', '883cf2c605fffff', '883cf2c607fffff', '883cf2c609fffff', '883cf2c60bfffff', '883cf2c60dfffff', '883cf2c621fffff', '883cf2c623fffff', '883cf2c625fffff', '883cf2c627fffff', '883cf2c629fffff', '883cf2c62bfffff', '883cf2c62dfffff', '883cf2c641fffff', '883cf2c643fffff', '883cf2c645fffff', '883cf2c647fffff', '883cf2c649fffff', '883cf2c64bfffff', '883cf2c64dfffff', '883cf2c661fffff', '883cf2c663fffff', '883cf2c665fffff', '883cf2c667fffff', '883cf2c669fffff', '883cf2c66bfffff', '883cf2c66dfffff', '883cf2c70bfffff', '883cf2c719fffff', '883cf2c71bfffff', '883cf2c71dfffff', '883cf2c741fffff', '883cf2c743fffff', '883cf2c749fffff', '883cf2c74bfffff', '883cf2c751fffff', '883cf2c753fffff', '883cf2c755fffff', '883cf2c757fffff', '883cf2c759fffff', '883cf2c75bfffff', '883cf2c75dfffff', '883cf2d4a7fffff', '883cf2d593fffff', '883cf2d59bfffff', '883cf2d5b5fffff', '883cf2d5b7fffff']


In [71]:
print(len(jaipur_hexes))

50


In [22]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Jaipur')
and resolution = 8
and hex_id in ('883da20a61fffff', '883da20a65fffff', '883da20a69fffff', '883da20a6dfffff', '883da20b59fffff', '883da21837fffff', '883da218a1fffff',
 '883da218a5fffff', '883da218a7fffff', '883da218adfffff', '883da21981fffff', '883da21983fffff', '883da21987fffff', '883da21989fffff', '883da2198bfffff',
  '883da2198dfffff', '883da21991fffff', '883da21995fffff', '883da21997fffff', '883da21999fffff', '883da2199bfffff', '883da2199dfffff', '883da219b1fffff',
   '883da219b3fffff', '883da219b7fffff', '883da219b9fffff', '883da219bbfffff', '883da219bdfffff', '883da219c1fffff', '883da219c3fffff', '883da219c5fffff',
    '883da219c7fffff', '883da219cbfffff', '883da219d1fffff', '883da219d3fffff', '883da219d5fffff', '883da219d7fffff', '883da219d9fffff', '883da219dbfffff',
     '883da219ddfffff', '883da219ebfffff')
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [72]:
jaipur_hexes = ['883da20a61fffff', '883da20a65fffff', '883da20a69fffff', '883da20a6dfffff', '883da20b59fffff', '883da21837fffff', '883da218a1fffff',
 '883da218a5fffff', '883da218a7fffff', '883da218adfffff', '883da21981fffff', '883da21983fffff', '883da21987fffff', '883da21989fffff', '883da2198bfffff',
  '883da2198dfffff', '883da21991fffff', '883da21995fffff', '883da21997fffff', '883da21999fffff', '883da2199bfffff', '883da2199dfffff', '883da219b1fffff',
   '883da219b3fffff', '883da219b7fffff', '883da219b9fffff', '883da219bbfffff', '883da219bdfffff', '883da219c1fffff', '883da219c3fffff', '883da219c5fffff',
    '883da219c7fffff', '883da219cbfffff', '883da219d1fffff', '883da219d3fffff', '883da219d5fffff', '883da219d7fffff', '883da219d9fffff', '883da219dbfffff',
     '883da219ddfffff', '883da219ebfffff']
jaipur_tam = pd.DataFrame(jaipur_hexes)
jaipur_tam.to_csv('haipur_tam.csv', index = False)

In [23]:
df_cluster_copy['cluster'].value_counts()

Gopalpura_JAI                 8
Malviya_Nagar_JAI 2           5
Mansarovar 2_JAI              3
Nirman_Nagar_JAI              2
Sudarshanpura_JAI             2
Malviya Nagar_JAI             2
Maharani Farm_JAI             2
MNIT_JAI                      2
Durgapura_JAI                 2
                              2
Swej Farm_JAI                 2
Satkar_Malviya_Nagar_JAI      1
Triveni_Nagar_JAI             1
VT Road Mansarovar_JAI        1
Jhalana Dungari_JAI           1
Lalkothi_JAI                  1
Narayan Vihar_JAI             1
Landscape_Park_JAI            1
Swarna Path Mansarovar_JAI    1
Gurjar_Ki_Thadi_JAI           1
Name: cluster, dtype: int64

In [60]:
df_level_1_grouped[df_level_1_grouped['hour'].isin(['08', '09', '10', '11', '17', '18', '19', '20', '21'])]

,date,hour,rides_per_hour,rides_per_20_sec
8,06,08,409,2.27
9,06,09,877,4.87
10,06,10,976,5.42
11,06,11,1003,5.57
17,06,17,1418,7.88
18,06,18,1796,9.98
19,06,19,1741,9.67
20,06,20,1234,6.86
21,06,21,812,4.51
32,07,08,209,1.16


In [18]:
df_level_1_grouped.groupby(['hour'])['rides_per_20_sec'].median()

hour
00    0.38
01    0.23
02    0.17
03    0.16
04    0.73
05    1.46
06    1.02
07    1.38
08    2.18
09    4.96
10    4.12
11    3.29
12    3.76
13    3.79
14    3.56
15    3.57
16    3.56
17    3.73
18    4.57
19    4.21
20    3.13
21    2.37
22    1.59
23    0.76
Name: rides_per_20_sec, dtype: float64

In [19]:
df_level_1_grouped.groupby(['hour'])['rides_per_20_sec'].mean()

hour
00    0.388
01    0.224
02    0.170
03    0.152
04    0.656
05    1.504
06    1.020
07    1.366
08    2.252
09    4.652
10    4.164
11    3.356
12    3.716
13    3.892
14    3.974
15    3.574
16    3.478
17    3.590
18    4.510
19    4.254
20    3.348
21    2.420
22    1.714
23    0.766
Name: rides_per_20_sec, dtype: float64

In [44]:
df_hex_high_demand_grouped.to_csv('demand_chennai.csv', index = False)

In [43]:
df_hex_high_demand_grouped = df_hex_high_demand.groupby(['customer_location_hex_8'])['count'].sum().reset_index()
df_hex_high_demand_grouped

,customer_location_hex_8,count
0,88618c4881fffff,1095
1,88618c4883fffff,3407
2,88618c4885fffff,3705
3,88618c4887fffff,1339
4,88618c4889fffff,2186
5,88618c488bfffff,1321
6,88618c488dfffff,369
7,88618c4891fffff,1183
8,88618c4893fffff,816
9,88618c4895fffff,1343
